# Load dataset and model

In [26]:
from util.get_dataset_model import get_dataset_model
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model, dataloader = get_dataset_model()
model.to(device)
x_batch, y_batch = iter(dataloader).next()
x_batch = x_batch.to(device)
y_batch = y_batch.to(device)

# Compute attributions

In [6]:
from captum.attr import Saliency

In [20]:
a_batch = Saliency(model).attribute(inputs=x_batch, target=y_batch, abs=True).sum(dim=1, keepdims=True).cpu().numpy()

/home/arne/Documents/PhD/projects/benchmark/benchmark/venv/lib/python3.8/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


# Compute metric scores

In [21]:
from attrbench.metrics import deletion
from attrbench.lib.masking import ConstantMasker

In [22]:
masker = ConstantMasker(feature_level="pixel")
del_results = deletion(x_batch, y_batch, model, a_batch, masker)

In [23]:
del_results

{'linear': tensor([[16.3768, 12.7624, 10.5919,  ...,  0.6784,  1.5342, -0.0694],
         [12.1573,  9.5815,  8.3827,  ..., -1.9531, -1.9773, -2.7090],
         [17.4794, 10.7758, 10.0164,  ...,  0.4730,  0.9697, -0.0694],
         ...,
         [17.7253, 12.0365, 11.7768,  ..., -1.1032, -1.0498, -1.9126],
         [17.1633, 12.6065, 11.9246,  ..., -1.3088, -1.4596, -1.9126],
         [19.0699, 17.5840, 17.6664,  ...,  2.6356,  1.8361,  2.3229]])}

In [24]:
del_results["linear"].shape

torch.Size([64, 100])